In [ ]:
! echo "::group::Install Dependencies"
%pip install uv
! uv pip install "git+https://github.com/ibm-granite-community/utils.git" \
    langgraph \
    langchain \
    langchain_ibm
! echo "::endgroup::"


In [20]:
from ibm_granite_community.notebook_utils import get_env_var
from langchain_core.utils.utils import convert_to_secret_str
from langchain.chat_models import init_chat_model

model = "ibm/granite-4-h-small"

model_parameters = {
    "temperature": 0,
    "max_completion_tokens": 500,
    "repetition_penalty": 1.05,
}

llm = init_chat_model(
    model=model,
    model_provider="ibm",
    url=convert_to_secret_str(get_env_var("WATSONX_URL")),
    apikey=convert_to_secret_str(get_env_var("WATSONX_APIKEY")),
    project_id=get_env_var("WATSONX_PROJECT_ID"),
    params=model_parameters,
)

In [22]:
#definte mock tools for now as test
from langchain.tools import tool

@tool
def get_weather(destination: str, month: str) -> str:
    """Get weather for a destination and month."""
    return f"The weather in {destination} in {month} is cool and dry, highs ~15°C."

@tool
def get_flight_info(destination: str, month: str) -> str:
    """Get flight cost for a destination and month."""
    return f"Flights to {destination} in {month} average around $1200 USD round-trip."

@tool
def get_hotel_prices(destination: str, month: str) -> str:
    """Get hotel prices for a destination and month."""
    return f"Hotels in {destination} range from $100 to $300 per night in {month}."

tools = [get_weather, get_flight_info, get_hotel_prices]


In [28]:
#we now want to go back to the from langchain.agents import create_agent

from langgraph.prebuilt import create_react_agent 

agent = create_react_agent(
    model=llm,
    tools=tools
)


/var/folders/tt/jz8hl4bx1qs6c7ljdflw7z9r0000gn/T/ipykernel_29986/773479898.py:3: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


In [24]:
response = agent.invoke({
    "messages": [{"role": "user", "content": "I want to visit Tokyo next month, which will be April. What should I expect in terms of weather and cost?"}]
})


In [27]:
final_message = response["messages"][-1].content
print(final_message)


**Tokyo in April – What to Expect**

| Category | Details |
|----------|---------|
| **Weather** | • **Temperature:** Cool and mild. Average highs around **15 °C (59 °F)**, lows near **8 °C (46 °F)**.<br>• **Precipitation:** Generally dry; only a few light showers expected. <br>• **What to Pack:** Light jacket or sweater for evenings, comfortable layers for daytime, and an umbrella just in case. |
| **Flights** | • **Average Round‑Trip Cost:** **≈ $1,200 USD** (prices can vary by airline, departure city, and how far in advance you book). |
| **Hotels** | • **Price Range:** **$100 – $300 USD per night** for most mid‑range hotels. <br>• **Budget Options:** Capsule hotels or budget guesthouses can be found for **$50 – $100 USD** per night.<br>• **Luxury Options:** High‑end hotels and ryokans start around **$250 USD** and can go well above **$500 USD** per night. |

### Quick Tips for Your April Trip

1. **Book Flights Early** – April is a popular time (cherry‑blossom season), so securing 